# Motion Prediction with PointNet

# About

In this work, I will be using the [pointnet architecture](https://github.com/charlesq34/pointnet) to predict autonomous vehicle motions. This work is completely different from the main public kernels and exposes another way of dealing with motion prediction.

One of the edges of the pointnet architecture is that it works directly on the agents set and, therefore, no rastarization is required. Hence, we can can get a decent accuracy  while training only on the small train set and just for 5 to 7 hours. Hence, this model could be run even if one doesn't have huge GPU capacities. But, if you do have enough GPU, you can increase the model size and train on the full train set.

### This kernel is inference only, the training process is heavier and is on its road !

The PointNet model uses a combination of feed forward (conv1d) models along with somme transformation matrices and symmetric functions (max pooling) to deal with unordered set of points (agents). For more reading, please visit the [PointNet project page](https://stanford.edu/~rqi/pointnet/) or [the original paper](https://arxiv.org/pdf/1612.00593.pdf). My code is inspired from [this github repos](https://github.com/charlesq34/pointnet).

![](https://stanford.edu/~rqi/pointnet/images/pointnet.jpg)

> PointNet can be applied for classification and image segmentation. Since we will be needing an output for each point (agent), only the segmentation version will be suitable.


![PointNet architecture](https://github.com/charlesq34/pointnet/blob/master/doc/teaser.png?raw=true)

In [ ]:
!pip install zarr > /dev/null
!pip install  --use-feature=2020-resolver pytorch-lightning  > /dev/null

In [ ]:
import zarr
from abc import ABC
from pathlib import Path
from numcodecs import blosc
import pandas as pd, numpy as np

import bisect
import itertools as it
from tqdm.notebook import tqdm


import torch
from torch import nn, optim 
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import pickle, copy, re,time, datetime, random, warnings

pd.options.display.max_columns=305

In [ ]:
# blosc.set_nthreads(6)  
# blosc.use_threads = True

In [ ]:
DATA_ROOT = Path("../input/lyft-motion-prediction-autonomous-vehicles")
ZARR_PATH = Path("scenes/test.zarr")
print("DATA_ROOT: {}\nZARR_PATH: {}".format(DATA_ROOT, ZARR_PATH))

In [ ]:
HBACKWARD = 15
HFORWARD = 0
NFRAMES = 1
FRAME_STRIDE = 0
AGENT_FEATURE_DIM = 8
MAX_AGENTS = 150

In [ ]:
TIME_FORMAT = r"%Y-%m-%dT%H:%M:%S%Z"
def get_utc():
    return datetime.datetime.now(datetime.timezone.utc).strftime(TIME_FORMAT)

# Dataset

In [ ]:
PERCEPTION_LABELS = [
    "PERCEPTION_LABEL_NOT_SET",
    "PERCEPTION_LABEL_UNKNOWN",
    "PERCEPTION_LABEL_DONTCARE",
    "PERCEPTION_LABEL_CAR",
    "PERCEPTION_LABEL_VAN",
    "PERCEPTION_LABEL_TRAM",
    "PERCEPTION_LABEL_BUS",
    "PERCEPTION_LABEL_TRUCK",
    "PERCEPTION_LABEL_EMERGENCY_VEHICLE",
    "PERCEPTION_LABEL_OTHER_VEHICLE",
    "PERCEPTION_LABEL_BICYCLE",
    "PERCEPTION_LABEL_MOTORCYCLE",
    "PERCEPTION_LABEL_CYCLIST",
    "PERCEPTION_LABEL_MOTORCYCLIST",
    "PERCEPTION_LABEL_PEDESTRIAN",
    "PERCEPTION_LABEL_ANIMAL",
    "AVRESEARCH_LABEL_DONTCARE",
]
KEPT_PERCEPTION_LABELS = [
    "PERCEPTION_LABEL_UNKNOWN",
    "PERCEPTION_LABEL_CAR",
    "PERCEPTION_LABEL_CYCLIST",
    "PERCEPTION_LABEL_PEDESTRIAN",
]
KEPT_PERCEPTION_LABELS_DICT = {label:PERCEPTION_LABELS.index(label) for label in KEPT_PERCEPTION_LABELS}
KEPT_PERCEPTION_KEYS = sorted(KEPT_PERCEPTION_LABELS_DICT.values())

In [ ]:
class LabelEncoder:
    def  __init__(self, max_size=500, default_val=-1):
        self.max_size = max_size
        self.labels = {}
        self.default_val = default_val

    @property
    def nlabels(self):
        return len(self.labels)

    def reset(self):
        self.labels = {}

    def partial_fit(self, keys):
        nlabels = self.nlabels
        available = self.max_size - nlabels

        if available < 1:
            return

        keys = set(keys)
        new_keys = list(keys - set(self.labels))

        if not len(new_keys):
            return
        
        self.labels.update(dict(zip(new_keys, range(nlabels, nlabels + available) )))
    
    def fit(self, keys):
        self.reset()
        self.partial_fit(keys)

    def get(self, key):
        return self.labels.get(key, self.default_val)
    
    def transform(self, keys):
        return np.array(list(map(self.get, keys)))

    def fit_transform(self, keys, partial=True):
        self.partial_fit(keys) if partial else self.fit(keys)
        return self.transform(keys)

In [ ]:
class CustomLyftDataset(Dataset):
    feature_mins = np.array([-17.336, -27.137, 0. , 0., 0. , -3.142, -37.833, -65.583],
    dtype="float32")[None,None, None]

    feature_maxs = np.array([17.114, 20.787, 42.854, 42.138,  7.079,  3.142, 29.802, 35.722],
    dtype="float32")[None,None, None]



    def __init__(self, zdataset, scenes=None, nframes=10, frame_stride=15, hbackward=10, 
                 hforward=50, max_agents=150, agent_feature_dim=8):
        """
        Custom Lyft dataset reader.
        
        Parmeters:
        ----------
        zdataset: zarr dataset
            The root dataset, containing scenes, frames and agents
            
        nframes: int
            Number of frames per scene
            
        frame_stride: int
            The stride when reading the **nframes** frames from a scene
            
        hbackward: int
            Number of backward frames from  current frame
            
        hforward: int
            Number forward frames from current frame
        
        max_agents: int 
            Max number of agents to read for each target frame. Note that,
            this also include the backward agents but not the forward ones.
        """
        super().__init__()
        self.zdataset = zdataset
        self.scenes = scenes if scenes is not None else []
        self.nframes = nframes
        self.frame_stride = frame_stride
        self.hbackward = hbackward
        self.hforward = hforward
        self.max_agents = max_agents

        self.nread_frames = (nframes-1)*frame_stride + hbackward + hforward

        self.frame_fields = ['timestamp', 'agent_index_interval']

        self.agent_feature_dim = agent_feature_dim

        self.filter_scenes()
      
    def __len__(self):
        return len(self.scenes)

    def filter_scenes(self):
        self.scenes = [scene for scene in self.scenes if self.get_nframes(scene) > self.nread_frames]


    def __getitem__(self, index):
        return self.read_frames(scene=self.scenes[index])

    def get_nframes(self, scene, start=None):
        frame_start = scene["frame_index_interval"][0]
        frame_end = scene["frame_index_interval"][1]
        nframes = (frame_end - frame_start) if start is None else ( frame_end - max(frame_start, start) )
        return nframes


    def _read_frames(self, scene, start=None):
        nframes = self.get_nframes(scene, start=start)
        assert nframes >= self.nread_frames

        frame_start = scene["frame_index_interval"][0]

        start = start or frame_start + np.random.choice(nframes-self.nread_frames)
        frames = self.zdataset.frames.get_basic_selection(
            selection=slice(start, start+self.nread_frames),
            fields=self.frame_fields,
            )
        return frames
    

    def parse_frame(self, frame):
        return frame

    def parse_agent(self, agent):
        return agent

    def read_frames(self, scene, start=None,  white_tracks=None, encoder=False):
        white_tracks = white_tracks or []
        frames = self._read_frames(scene=scene, start=start)

        agent_start = frames[0]["agent_index_interval"][0]
        agent_end = frames[-1]["agent_index_interval"][1]

        agents = self.zdataset.agents[agent_start:agent_end]


        X = np.zeros((self.nframes, self.max_agents, self.hbackward, self.agent_feature_dim), dtype=np.float32)
        target = np.zeros((self.nframes, self.max_agents, self.hforward, 2),  dtype=np.float32)
        target_availability = np.zeros((self.nframes, self.max_agents, self.hforward), dtype=np.uint8)
        X_availability = np.zeros((self.nframes, self.max_agents, self.hbackward), dtype=np.uint8)

        for f in range(self.nframes):
            backward_frame_start = f*self.frame_stride
            forward_frame_start = f*self.frame_stride+self.hbackward
            backward_frames = frames[backward_frame_start:backward_frame_start+self.hbackward]
            forward_frames = frames[forward_frame_start:forward_frame_start+self.hforward]

            backward_agent_start = backward_frames[-1]["agent_index_interval"][0] - agent_start
            backward_agent_end = backward_frames[-1]["agent_index_interval"][1] - agent_start

            backward_agents = agents[backward_agent_start:backward_agent_end]

            le = LabelEncoder(max_size=self.max_agents)
            le.fit(white_tracks)
            le.partial_fit(backward_agents["track_id"])

            for iframe, frame in enumerate(backward_frames):
                backward_agent_start = frame["agent_index_interval"][0] - agent_start
                backward_agent_end = frame["agent_index_interval"][1] - agent_start

                backward_agents = agents[backward_agent_start:backward_agent_end]

                track_ids = le.transform(backward_agents["track_id"])
                mask = (track_ids != le.default_val)
                mask_agents = backward_agents[mask]
                mask_ids = track_ids[mask]
                X[f, mask_ids, iframe, :2] = mask_agents["centroid"]
                X[f, mask_ids, iframe, 2:5] = mask_agents["extent"]
                X[f, mask_ids, iframe, 5] = mask_agents["yaw"]
                X[f, mask_ids, iframe, 6:8] = mask_agents["velocity"]

                X_availability[f, mask_ids, iframe] = 1

            
            for iframe, frame in enumerate(forward_frames):
                forward_agent_start = frame["agent_index_interval"][0] - agent_start
                forward_agent_end = frame["agent_index_interval"][1] - agent_start

                forward_agents = agents[forward_agent_start:forward_agent_end]

                track_ids = le.transform(forward_agents["track_id"])
                mask = track_ids != le.default_val

                target[f, track_ids[mask], iframe] = forward_agents[mask]["centroid"]
                target_availability[f, track_ids[mask], iframe] = 1

        target -= X[:,:,[-1], :2]
        target *= target_availability[:,:,:,None]
        X[:,:,:, :2] -= X[:,:,[-1], :2]
        X *= X_availability[:,:,:,None]
        X -= self.feature_mins
        X /= (self.feature_maxs - self.feature_mins)

        if encoder:
            return X, target, target_availability, le
        return X, target, target_availability

# PointNet

In [ ]:
class STNkd(nn.Module):
    def __init__(self,  k=64):
        super(STNkd, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv1d(k, 256, kernel_size=1), nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=1), nn.ReLU(),
            nn.Conv1d(256, 512, kernel_size=1), nn.ReLU(),
        )
        
        self.fc = nn.Sequential(
            nn.Linear(512, k*k),nn.ReLU(),
        )
        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = self.conv(x)
        x = torch.max(x, 2)[0]
        x = self.fc(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,
                                                                            self.k*self.k).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

In [ ]:
class PointNetfeat(nn.Module):
    def __init__(self, global_feat = False, feature_transform = False, stn1_dim = 120,
                 stn2_dim = 64):
        super(PointNetfeat, self).__init__()
        
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        self.stn1_dim = stn1_dim
        self.stn2_dim = stn2_dim
        
        self.stn = STNkd(k=stn1_dim)
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(stn1_dim, 256, kernel_size=1), nn.ReLU(),
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=1), nn.ReLU(),
            nn.Conv1d(256, 1024, kernel_size=1), nn.ReLU(),
            nn.Conv1d(1024, 2048, kernel_size=1), nn.ReLU(),
        )
        
        if self.feature_transform:
            self.fstn = STNkd(k=stn2_dim)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        
        x = self.conv1(x)

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        
        x = self.conv2(x)
        x = torch.max(x, 2)[0]
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x[:,:,None].repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat

In [ ]:
class LyftNet(nn.Module):   
    def __init__(self):
        super().__init__()
        
        self.pnet = PointNetfeat()

        self.fc0 = nn.Sequential(
            nn.Linear(2048+256, 1024), nn.ReLU(),
        )

        self.fc = nn.Sequential(
            nn.Linear(1024, 300),
        )

        self.c_net = nn.Sequential(
            nn.Linear(1024, 3),
        )
        
    
    def forward(self, x):
        bsize, npoints, hb, nf = x.shape 
        
        # Push points to the last  dim
        x = x.transpose(1, 3)

        # Merge time with features
        x = x.reshape(bsize, hb*nf, npoints)

        x, trans, trans_feat = self.pnet(x)

        # Push featuresxtime to the last dim
        x = x.transpose(1,2)

        x = self.fc0(x)

        c = self.c_net(x)
        x = self.fc(x)

        return c,x

# Prediction

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
%%time

mask = np.load(DATA_ROOT/"scenes/mask.npz")['arr_0']
agent_ids = np.where(mask)[0]
agent_ids

In [ ]:
z = zarr.open(DATA_ROOT.joinpath(ZARR_PATH).as_posix())
frames_ij = z.scenes["frame_index_interval"]
agents_ij = z.frames["agent_index_interval"]

In [ ]:
def get_scene(agent_id):
    frame_id = bisect.bisect_right(agents_ij[:, 0], agent_id)-1
    scene_id = bisect.bisect_right(frames_ij[:, 0], frame_id)-1
    
    scene = z.scenes[scene_id]
    frame = z.frames[frame_id]
    agent = z.agents[agent_id]
    return scene,(frame,frame_id),agent

In [ ]:
agent_id = agent_ids[np.random.choice(len(agent_ids))]
scene,(frame,frame_id), agent = get_scene(agent_id)
scene,(frame,frame_id), agent["track_id"]

In [ ]:
dt = CustomLyftDataset(
        z, 
        nframes=NFRAMES,
        frame_stride=FRAME_STRIDE,
        hbackward=HBACKWARD,
        hforward=HFORWARD,
        max_agents=MAX_AGENTS,
        agent_feature_dim=AGENT_FEATURE_DIM,
)
dt.nread_frames

In [ ]:
paths = [
    "../input/neural-net-on-lyft-tabular-data/lyfnet_epoch13.ckpt",
]
weights = np.array([1.0])
weights /= weights.sum()
nets = []
for path in paths:
    net = LyftNet().to(DEVICE)
    net.load_state_dict(torch.load(path, map_location=DEVICE)["state_dict"])
    net.eval()
    nets.append(net)

nets[0]

In [ ]:
def make_colnames():
    cols= list(
        it.chain(*[
    [f"coord_x0{i}", f"coord_y0{i}", f"coord_x1{i}", f"coord_y1{i}", f"coord_x2{i}", f"coord_y2{i}"]
    for i in range(50)]))
    return ["timestamp", "track_id"] + ["conf_0", "conf_1", "conf_2"] + cols

In [ ]:
@torch.no_grad()
def predict(agent_id):
    scene, (frame,frame_id), agent = get_scene(agent_id)
    c = np.zeros(3)
    y = np.zeros(300)
    try:
        X, _,_, le = dt.read_frames(scene,
            start=frame_id-HBACKWARD+1,
            white_tracks=[agent["track_id"]],
            encoder=True
        )
        
        X = torch.from_numpy(X).to(DEVICE)
        cs,ys=  [],[]
        for net in nets:
            c, y = net(X)
            c = torch.softmax(c[0, [le.labels[agent["track_id"]]]],dim=1)[0].cpu().numpy()
            y = y[0, le.labels[agent["track_id"]]].cpu().numpy()
            cs.append(c)
            ys.append(y)
            
        cs = np.sum(np.vstack(cs)*weights[None], axis=1)
        ys = np.vstack(ys).reshape(len(ys), 50, 3, 2)
        ys = np.sum(ys*weights[None,None,:,None], axis=2).reshape(len(ys), -1)
        
    except Exception as e:
        print(f'Exception : {e}')
#         raise ValueError() from e
    return [frame["timestamp"]],[agent["track_id"]],c,y

In [ ]:
%%time

timestamp,track_id, c,y = [],[],[],[]

for icount, agent_id in tqdm(list(enumerate(agent_ids))):
    _timestamp, _track_id, _c,_y = predict(agent_id)
    timestamp.extend(_timestamp)
    track_id.extend(_track_id)
    c.append(_c)
    y.append(_y)
c = np.vstack(c)
y = np.vstack(y)
y.shape, c.shape

In [ ]:
%%time

cols = make_colnames()
df = pd.DataFrame(columns=cols)
df[cols[0]] = timestamp
df[cols[1]] = track_id
df[cols[2:5]] = c
df[cols[5:]] = y
df.shape

In [ ]:
df.head(20)

In [ ]:
df.to_csv("submission.csv", index=False)

<div style="text-align: center"><strong>Thanks</strong></div>